In [6]:
# Libraries
import pandas as pd
import numpy as np
import vcf
#

#    "Thyroid": "green",
SHORTEN = {
    "Artery-Aorta":"Artery.A"     ,
    "Artery-Tibial": "Artery.T",
    "Adipose-Subcutaneous": "Adipose.S",    
    "Adipose-Visceral(Omentum)":"Adipose.V",
    "Brain-Caudate(basalganglia)":"Caudate"   , 
    "Brain-Cerebellum":"Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "Mucosa",
    "Esophagus-Muscularis":"Muscularis",
    "Heart-LeftVentricle":"Heart",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve",
    "Skin-NotSunExposed(Suprapubic)": "SkinUnexposed",
    "Skin-SunExposed(Lowerleg)":"SkinLeg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood",
    "permuted":"Permuted",
}

GENE = '/storage/szfeupe/Runs/GTEx_estr/FEATURES/Genes_only_table'
PATH= "/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/"
TISSUES = sorted([item for item in list(SHORTEN.keys()) if item != "permuted"])
motif='/storage/resources/dbase/human/hg19/hg19.hipstr_reference_withmotif.bed.gz'

In [7]:
#Linear regression QVALUES
#
GENES = pd.read_csv(GENE, sep='\t')
col=['chrom','str.start','str.id','gene']
SUMMARY = pd.read_csv(PATH +'WholeBlood/PQValues', sep = '\t')[col]
estrs = []
SUM = {}
Z = 0
#Merging by tissues 
for T in TISSUES:
    table = pd.read_csv(PATH +T+'/PQValues', sep = '\t')
    #table['eSTR.'+SHORTEN[T]+'.ll'] = np.where(table['llqvalue']<=0.1, 1, 0)  #10%fdr at locus level
    table['eSTR.'+SHORTEN[T]] = np.where(table['qvalue']<=0.1, 1, 0)
    table['qval.'+SHORTEN[T]] = table['qvalue'].astype(float)
    table['llqval.'+SHORTEN[T]] = table['llqvalue'].astype(float)
    table['beta.'+SHORTEN[T]] = table['beta'].astype(float)
    table = table[col + ['eSTR.'+SHORTEN[T], 'qval.'+SHORTEN[T],'llqval.'+SHORTEN[T],'beta.'+SHORTEN[T]]]
    table['str.start'].astype(int)
    table['chrom'].astype(str)
    #estrs.append('eSTR.'+SHORTEN[T])
    SUM[T]=table
    table['eSTR.'+SHORTEN[T]] = table['eSTR.'+SHORTEN[T]].astype(int)
    SUMMARY = pd.merge(SUMMARY,table[col], on=col, how='outer').drop_duplicates().reset_index(drop=True)
    print(table.shape[0],'\t',SUMMARY.shape,'\t',table.loc[table['eSTR.'+SHORTEN[T]]==1].shape[0],'\t', T)
    Z=Z+table.shape[0]
SUMMARY = SUMMARY.sort_values(['chrom','gene', 'str.start']).reset_index(drop=True)
SUMMARY['E.tissues']=[0]*SUMMARY.shape[0]
SUMMARY['ll.E.tissues']=[0]*SUMMARY.shape[0]
for T in TISSUES :
    print (T)
    SUMMARY = pd.merge(SUMMARY.fillna(0), SUM[T], how='left',on=col)
    SUMMARY['E.tissues'] = SUMMARY['E.tissues'] + SUMMARY['eSTR.'+SHORTEN[T]].fillna(0).astype(int)


print('Number of egenes... ',SUMMARY.loc[SUMMARY['E.tissues']>=1].shape)
print('Nbr of eGenes in more than 1 tissues ',SUMMARY.loc[SUMMARY['E.tissues']>=2].shape)
print('Nbr of eGenes in at least 5 tissues ',SUMMARY.loc[SUMMARY['E.tissues']>=5].shape)
print('Nbr of eGenes in at least 10 tissues ',SUMMARY.loc[SUMMARY['E.tissues']>=10].shape)

add_info = pd.read_csv(motif, sep='\t', header=None)
add_info.columns=['chrom','str.start','str.end','period','motif']
print(add_info.shape)
SUMMARY = pd.merge(SUMMARY, add_info, on=['chrom','str.start'], how='inner')
e_homopolymers = SUMMARY.loc[(SUMMARY['E.tissues']>=1) & (SUMMARY['period']==1)]
print(SUMMARY.shape, e_homopolymers.shape)

SUMMARY.to_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/LR_SummaryTest_Table.tsv', sep='\t', index=False)


281318 	 (286612, 4) 	 3596 	 Adipose-Subcutaneous
281223 	 (290076, 4) 	 1842 	 Adipose-Visceral(Omentum)
277044 	 (292265, 4) 	 2401 	 Artery-Aorta
275588 	 (292752, 4) 	 3462 	 Artery-Tibial
278997 	 (299484, 4) 	 682 	 Brain-Caudate(basalganglia)
277449 	 (301908, 4) 	 1555 	 Brain-Cerebellum
267909 	 (302822, 4) 	 3096 	 Cells-Transformedfibroblasts
281861 	 (304085, 4) 	 3468 	 Esophagus-Mucosa
280638 	 (304403, 4) 	 2831 	 Esophagus-Muscularis
271269 	 (304722, 4) 	 1733 	 Heart-LeftVentricle
286473 	 (305270, 4) 	 2992 	 Lung
271694 	 (305429, 4) 	 3373 	 Muscle-Skeletal
284926 	 (305757, 4) 	 4364 	 Nerve-Tibial
285850 	 (306652, 4) 	 2393 	 Skin-NotSunExposed(Suprapubic)
285435 	 (306693, 4) 	 3836 	 Skin-SunExposed(Lowerleg)
285470 	 (307187, 4) 	 4113 	 Thyroid
261714 	 (307187, 4) 	 2587 	 WholeBlood
Adipose-Subcutaneous
Adipose-Visceral(Omentum)
Artery-Aorta
Artery-Tibial
Brain-Caudate(basalganglia)
Brain-Cerebellum
Cells-Transformedfibroblasts
Esophagus-Mucosa
Esophagus-

In [11]:
#SUMMARY.loc[SUMMARY['E.tissues']>=1]
8152*100/25619

31.82013349467192

In [4]:
#Number of eSTRs by STR unit size
sumtable = {}
sumtable['Unit.size'] = [x for x in range(1,7,1)]
sumtable['all.tissues'] = [ SUMMARY.loc[(SUMMARY['E.tissues']>=1) & (SUMMARY['period']==i)].shape[0] for i in range(1,7,1)]

for T in TISSUES:
    sumtable[SHORTEN[T]] = [ SUMMARY.loc[(SUMMARY['eSTR.'+SHORTEN[T]]>=1) & (SUMMARY['period']==i)].shape[0] for i in range(1,7,1)]
    
estr_by_unit = pd.DataFrame(sumtable)[['Unit.size', 'all.tissues']+ [SHORTEN[T] for T in TISSUES]]
estr_by_unit.index = list(estr_by_unit['Unit.size'])
estr_by_unit['Unit.size']

estr_by_unit

,Unit.size,all.tissues,Adipose.S,Adipose.V,Artery.A,Artery.T,Caudate,Cerebellum,Fibroblast,Mucosa,Muscularis,Heart,Lung,Muscle,Nerve,SkinUnexposed,SkinLeg,Thyroid,Blood
1,1,10250,1357,713,971,1371,269,588,1246,1363,1149,685,1199,1318,1690,956,1491,1619,1033
2,2,7037,959,493,625,902,179,441,807,900,728,458,801,861,1142,632,1014,1099,634
3,3,1609,258,132,149,238,49,114,209,240,193,114,217,225,316,161,293,280,191
4,4,4690,679,351,429,637,118,280,553,676,526,314,519,671,842,438,731,781,485
5,5,1331,228,106,152,206,48,90,183,179,151,117,179,183,259,138,204,214,156
6,6,702,115,47,75,108,19,42,98,110,84,45,77,115,115,68,103,120,88


In [12]:
E = SUMMARY.loc[SUMMARY['E.tissues']>=1]
U = SUMMARY.loc[SUMMARY['E.tissues']==1]
S = SUMMARY.loc[SUMMARY['E.tissues']>1]
E1 = E.loc[E['period']==1]
print('Total # eSTRs',E.shape[0],'\nUnique to tissues', U.shape[0],'\nShared eSTRs', S.shape[0])
H = U.loc[U['period']==1]; S1 = S.loc[S['period']==1]
print('Total # of Unique to tissue eSTRs',U.shape[0],'\nNumber of homopolymers ', H.shape[0],'\n Shared homopolymers eSTRs', S1.shape[0])

E1.shape

Total # eSTRs 25619 
Unique to tissues 17467 
Shared eSTRs 8152
Total # of Unique to tissue eSTRs 17467 
Number of homopolymers  7013 
 Shared homopolymers eSTRs 3237


(10250, 77)